In [91]:
import numpy as np
from scipy.optimize import fsolve

# Rocket engine thrust as described in 
# https://www.grc.nasa.gov/www/k-12/airplane/rktthsum.html


p_t = 30000   # pressure in tank [kPa]
T_t = 300     # temperature in tank [K]
d_star = 0.04 # diameter at throad [m]
d_e = .6      # diameter at exit [m]

gam = 1.4     # specific heat ratio for oxygen
R = 0.286     # specific gas constant for oxygen [kJ/kg/K]

p_0 = 0.86    # free stream pressure [kPa]

A_e = np.pi * (d_e/2)**2
A_star = np.pi * (d_star/2)**2


m_dot = (A_star * p_t / np.sqrt(T_t)) * np.sqrt(gam/R) * ((gam+1.)/2.)**(-((gam+1.)/(gam-1.)/2.))
M_e = fsolve(
    lambda M_e: (((gam+1.)/2.)**(-((gam+1.)/(gam-1.)/2))) / M_e * (1. + M_e**2 * (gam-1.)/2.)**((gam+1.)/(gam-1.)/2.) - A_e/A_star
    , 1.)[0]
p_e = p_t * (1. + M_e**2. * (gam-1.)/2.)**(-(gam/(gam-1.)))
T_e = T_t * (1. + M_e**2. * (gam-1.)/2.)**(-1)
v_e = M_e * np.sqrt(gam * R * T_e)
F = m_dot * v_e + (p_e - p_0) * A_e

print """
Thrust: {} [N]
m_dot:  {} [kg/s]
M_e:    {} 
p_e:    {} [kPa]
T_e:    {} [K]
v_e:    {} [m/s]
""".format(F, m_dot, M_e, p_e, T_e, v_e)


Thrust: 4433.59790585 [N]
m_dot:  2.78681252989 [kg/s]
M_e:    1.01447994839 
p_e:    15581.7067315 [kPa]
T_e:    248.790481617 [K]
v_e:    10.1252879368 [m/s]

